### ESM-1: two generators, single load, single bus optimization problem.

In [349]:
import pypsa
import numpy as np
import pandas as pd
import linopy

In [376]:
n = pypsa.Network()

In [377]:
n

Empty PyPSA Network
Components: none
Snapshots: 1

In [378]:
# 48.3794° N, 31.1656° E
n.add("Bus", "UA", v_nom=380)

In [379]:
n.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
UA,380.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,,


In [380]:
n.add(
    "Generator",
    "gas",
    bus="UA",
    p_nom_extendable=False,
    marginal_cost=70,  # €/MWh
    p_nom=50,  # MW
)
n.add(
    "Generator",
    "coal",
    bus="UA",
    p_nom_extendable=False,
    marginal_cost=40,  # €/MWh
    p_nom=100,  # MW
)

In [381]:
n.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gas,UA,PQ,,50.0,False,0.0,inf,0.0,1.0,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
coal,UA,PQ,,100.0,False,0.0,inf,0.0,1.0,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [382]:
# your code
n.add(
    "Load",
    "demand",
    bus="UA",
    p_set=pd.Series([120], index=["now"]),  # MW
)

In [383]:
n.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
demand,UA,,,0.0,0.0,-1.0


### Solve

In [384]:
n.optimize(solver_name="glpk")

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 5.40e+03
Solver model: not available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-g38s06y5.lp --output /tmp/linopy-solve-jh1i8amd.sol
Reading problem data from '/tmp/linopy-problem-g38s06y5.lp'...
5 rows, 2 columns, 6 non-zeros
35 lines were read
GLPK Simplex Optimizer 5.0
5 rows, 2 columns, 6 non-zeros
Preprocessing...
~     0: obj =   5.400000000e+03  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/linopy-solve-jh1i8amd.sol'...


('ok', 'optimal')

In [385]:
# n.model.dual

### Solve problem manually 

In [386]:
# IPAD

### Explore pypsa model

In [310]:
n.optimize.create_model()

Linopy LP model

Variables:
----------
 * Generator-p (snapshot, Generator)

Constraints:
------------
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Bus-nodal_balance (Bus, snapshot)

Status:
-------
initialized

In [388]:
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: 5400.0

In [312]:
n.model.constraints

linopy.model.Constraints
------------------------
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Bus-nodal_balance (Bus, snapshot)

In [391]:
# n.model.constraints['Generator-fix-p-upper']

In [392]:
# n.model.constraints['Bus-nodal_balance']

# Reproduce linopy expressions

In [393]:
# reproduce objective
# n.model['Generator-p'].loc[:, 'gas'] * n.generators.marginal_cost.loc['gas'] + n.model['Generator-p'].loc[:, 'coal'] * n.generators.marginal_cost.loc['coal']

In [394]:
# reproduce ['Generator-fix-p-lower']
# n.model['Generator-p'].loc[:, 'gas'] >= 0
# n.model['Generator-p'].loc[:, 'coal'] >= 0

In [395]:
# reproduce ['Generator-fix-p-upper']
# n.model['Generator-p'].loc[:, 'gas'] <= n.generators.p_nom.loc['gas']
# n.model['Generator-p'].loc[:, 'coal'] <= n.generators.p_nom.loc['coal']

In [396]:
# reproduce ['Bus-nodal_balance']
# n.model['Generator-p'].sum(dims='Generator') == n.loads_t.p_set['demand']

# Rewrite pypsa default equations


In [399]:
for name in list(n.model.constraints):
    n.model.remove_constraints(name)

In [400]:
n.model.constraints

linopy.model.Constraints
------------------------
<empty>

In [401]:
n.model.objective.expression = linopy.LinearExpression(None, model=n.model)

In [402]:
n.model.objective.expression

LinearExpression
----------------
+0

### rewrite objective

In [398]:
(
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

LinearExpression (snapshot: 1):
-------------------------------
[now]: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]

In [403]:
n.model.objective.expression = (
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

In [404]:
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: 5400.0

### rewrite constraints

In [405]:
n.model.add_constraints(
    n.model["Generator-p"].sum(dims="Generator") == n.loads_t.p_set["demand"],
    name="nodal_balance",
)

Constraint `nodal_balance` (snapshot: 1):
-----------------------------------------
[now]: +1 Generator-p[now, gas] + 1 Generator-p[now, coal] = 120.0

In [406]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "gas"] >= 0, name="p_lower_gas")

Constraint `p_lower_gas` (snapshot: 1):
---------------------------------------
[now]: +1 Generator-p[now, gas] ≥ -0.0

In [407]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "coal"] >= 0, name="p_lower_coal")

Constraint `p_lower_coal` (snapshot: 1):
----------------------------------------
[now]: +1 Generator-p[now, coal] ≥ -0.0

In [408]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "gas"] <= n.generators.p_nom.loc["gas"],
    name="p_upper_gas",
)

Constraint `p_upper_gas` (snapshot: 1):
---------------------------------------
[now]: +1 Generator-p[now, gas] ≤ 50.0

In [410]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "coal"] <= n.generators.p_nom.loc["coal"],
    name="p_upper_coal",
)

Constraint `p_upper_coal` (snapshot: 1):
----------------------------------------
[now]: +1 Generator-p[now, coal] ≤ 100.0

In [413]:
n.model.constraints

linopy.model.Constraints
------------------------
 * nodal_balance (snapshot)
 * p_lower_gas (Generator, snapshot)
 * p_lower_coal (Generator, snapshot)
 * p_upper_gas (Generator, snapshot)
 * p_upper_coal (Generator, snapshot)

### re-solve

In [411]:
n.model.solve(solver_name="glpk")

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 5.40e+03
Solver model: not available
Solver message: optimal



GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-30yq1qi8.lp --output /tmp/linopy-solve-tdexfo7l.sol
Reading problem data from '/tmp/linopy-problem-30yq1qi8.lp'...
5 rows, 2 columns, 6 non-zeros
33 lines were read
GLPK Simplex Optimizer 5.0
5 rows, 2 columns, 6 non-zeros
Preprocessing...
~     0: obj =   5.400000000e+03  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/linopy-solve-tdexfo7l.sol'...


('ok', 'optimal')

In [412]:
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: 5400.0

In [414]:
n.generators_t.p

Generator,gas,coal
snapshot,,
now,20.0,100.0


## Hometask

In [417]:
n.optimize()

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 5.40e+03
Solver model: not available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-0a6twjg0.lp --output /tmp/linopy-solve-x8ho5e0a.sol
Reading problem data from '/tmp/linopy-problem-0a6twjg0.lp'...
5 rows, 2 columns, 6 non-zeros
35 lines were read
GLPK Simplex Optimizer 5.0
5 rows, 2 columns, 6 non-zeros
Preprocessing...
~     0: obj =   5.400000000e+03  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/linopy-solve-x8ho5e0a.sol'...


('ok', 'optimal')

In [420]:
n.model.dual

<xarray.Dataset>
Dimensions:                (snapshot: 1, Generator-fix: 2, Bus: 1)
Coordinates:
  * snapshot               (snapshot) object 'now'
  * Generator-fix          (Generator-fix) object 'gas' 'coal'
  * Bus                    (Bus) object 'UA'
Data variables:
    Generator-fix-p-lower  (snapshot, Generator-fix) float64 0.0 0.0
    Generator-fix-p-upper  (snapshot, Generator-fix) float64 0.0 -30.0
    Bus-nodal_balance      (snapshot, Bus) float64 70.0